In [4]:
%load_ext autoreload
%autoreload 2

import cv2
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path
import smplx  # SMPL/SMPLX model library
import sys
import torch

root_path = "/data/MHL/fifa-skeletal"
sys.path.append("../aitviewer")

from aitviewer.configuration import CONFIG as C

from lib.pitch import SoccerPitch
from lib.utils import project_points
from main import make_post_fk_func, create_smpl_sequences


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
def inspect_npz_file(file_path):
    # Open the .npz file using np.load
    with np.load(file_path) as data:
        print(f"File: {file_path}")
        # List all keys (array names) stored in the file
        keys = list(data.keys())
        print("Keys:", keys)
        # For each key, print the array's shape and data type
        for key in keys:
            array = data[key]
            print(f"Key: {key}, Shape: {array.shape}, Data type: {array.dtype}")
        print("-" * 40)

def print_video_info(cap):
    """
    Prints video information such as frame count, FPS, resolution, and duration.
    """
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    duration = frame_count / fps if fps else 0
    print("Video Information:")
    print(f" - Frame count: {frame_count}")
    print(f" - FPS: {fps}")
    print(f" - Resolution: {width} x {height}")
    print(f" - Duration: {duration:.2f} seconds")

clip_name = "ARG_CRO_220001"

data_dir = Path(root_path)
smpl_param_path = data_dir / f"poses/{clip_name}.npz"
video_path = data_dir / f"video_train/{clip_name}.mp4"
calibration_path = data_dir / f"cameras_train/{clip_name}.npz"

if not smpl_param_path.exists():
    raise FileNotFoundError(f"SMPL parameters not found at {smpl_param_path}")
if not video_path.exists():
    raise FileNotFoundError(f"Video not found at {video_path}")
if not calibration_path.exists():
    raise FileNotFoundError(f"Calibration parameters not found at {calibration_path}")

print("=== Inspecting Camera Data ===")
# inspect_npz_file(camera_file)
inspect_npz_file(calibration_path)

print("\n=== Inspecting Pose Data ===")
inspect_npz_file(smpl_param_path)

print("\n=== Video Information ===")
cap = cv2.VideoCapture(video_path)
print_video_info(cap)


=== Inspecting Camera Data ===
File: /data/MHL/fifa-skeletal/cameras_train/ARG_CRO_220001.npz
Keys: ['k', 'K', 'R', 't', 'Rt']
Key: k, Shape: (1032, 5), Data type: float64
Key: K, Shape: (1032, 3, 3), Data type: float64
Key: R, Shape: (1032, 3, 3), Data type: float64
Key: t, Shape: (1032, 3), Data type: float64
Key: Rt, Shape: (1032, 3, 4), Data type: float64
----------------------------------------

=== Inspecting Pose Data ===
File: /data/MHL/fifa-skeletal/poses/ARG_CRO_220001.npz
Keys: ['global_orient', 'body_pose', 'transl', 'betas']
Key: global_orient, Shape: (22, 1032, 3), Data type: float32
Key: body_pose, Shape: (22, 1032, 69), Data type: float32
Key: transl, Shape: (22, 1032, 3), Data type: float32
Key: betas, Shape: (22, 10), Data type: float32
----------------------------------------

=== Video Information ===
Video Information:
 - Frame count: 1032
 - FPS: 50.0
 - Resolution: 1920 x 1080
 - Duration: 20.64 seconds


In [5]:
# define constants
C.z_up = True
C.smplx_models = f"{root_path}/models"

camera_params = dict(np.load(calibration_path))

# Load SMPL parameters
smpl_params = dict(np.load(smpl_param_path))
colors = np.random.rand(len(smpl_params["betas"]), 4)
colors[:, 3] = 1

# Create a post-processing function (post_fk_func) to apply transformations to the vertices and joints
# The function uses camera parameters for additional transformations
post_fk_func = make_post_fk_func(camera_params)

# Create the SMPL sequences for each person using the SMPL parameters, colors, and post-processing function
# The SMPL sequences are stored in the list smpl_seqs
smpl_seqs = create_smpl_sequences(smpl_params, colors=colors, post_fk_func=post_fk_func)


In [14]:
# Check data

i = 0 # first frame
smpl_seq = smpl_seqs[i]
print(f"SMPL Sequence {i}:")

# Access vertices and joints
vertices = smpl_seq.vertices  # The vertices of the body mesh
joints = smpl_seq.joints  # The joints of the body
print(f"Vertices shape: {vertices.shape}")
print(f"Joints shape: {joints.shape}")

# # Print first few values of vertices and joints for inspection
# print(f"First 5 vertices:\n{vertices[:5]}")
# print(f"First 5 joints:\n{joints[:5]}")

# If you want to check other properties like mesh data, you can also print them
if hasattr(smpl_seq, 'mesh_seq'):
    print(f"Mesh sequence: {smpl_seq.mesh_seq}")

# Access the skeleton to understand the body joint connections
# The skeleton defines the parent-child relationships between joints.
# "body" refers to the skeletal structure of the body, excluding hands and face.
skeleton = smpl_seq.smpl_layer.skeletons()["body"] #all, body, hands

# Print the skeleton (parent-child relationships)
# The skeleton is represented as an array with pairs of parent and child joint indices.
# For example, skeleton[i] = [parent_index, child_index] means joint at parent_index is the parent of the joint at child_index.
print(f"Skeleton (body joint connections):\n{skeleton}")

print("-" * 40)

SMPL Sequence 0:
Vertices shape: (1032, 6890, 3)
Joints shape: (1032, 24, 3)
Mesh sequence: <aitviewer.renderables.meshes.Meshes object at 0x7f85a5e317b0>
Skeleton (body joint connections):
tensor([[-1,  0,  0,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9,  9,  9, 12, 13, 14,
         16, 17, 18, 19, 20, 21],
        [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19, 20, 21, 22, 23]], device='cuda:0')
----------------------------------------


In [15]:
PITCH = SoccerPitch()
soccer_pitch_3d_points = PITCH.pts  # 3D points of the soccer pitch keypoints
SOCCER_FIELD_LINES = PITCH.lines

## 

## 1. Genereate image with skeletal for one frame

In [24]:
print(f"{root_path}/outputs/{clip_name}/{frame_num:06d}.jpg")

/data/MHL/fifa-skeletal/outputs/ARG_CRO_220001/001032.jpg


In [20]:
# Define image path and frame number.
frame_num = 0
image_path = f"{root_path}/outputs/{clip_name}/{frame_num:06d}.jpg"

# Read image in BGR format
img = cv2.imread(image_path)

# Camera parameters
R = camera_params["R"][frame_num]
t = camera_params["t"][frame_num]
k = camera_params["k"][frame_num]
f = camera_params["K"][frame_num][0, 0]
principal_points = camera_params["K"][frame_num][:2, 2]
img_shape = img.shape

# Draw soccer field lines
for line in SOCCER_FIELD_LINES:
    line = project_points(line, R=R, t=t, f=f, principal_points=principal_points, k=k, img_shape=img_shape)
    line = line.astype(np.int32)
    cv2.polylines(img, [line], isClosed=False, color=(0, 0, 255), thickness=2)  # Red lines

# Visualize SMPL joints
for smpl_seq in smpl_seqs:
    joints = smpl_seq.joints[frame_num]  # frame's joint coordinates
    if np.isnan(joints).all():  # Skip if all joints are NaN
        continue

    # Project joints to 2D
    joints_2d = project_points(joints, R=R, t=t, f=f, principal_points=principal_points, k=k)
    
    # Draw joint positions
    for joint in joints_2d:
        x, y = joint.astype(int)
        cv2.circle(img, (x, y), 5, (0, 255, 0), -1)  # Green circles

    # Draw joint connections (skeleton)
    skeleton = smpl_seq.smpl_layer.skeletons()["body"]  # Body joint connections
    for parent_idx, child_idx in skeleton.T:
        if parent_idx == -1:  # Skip root joint
            continue

        parent_joint = joints_2d[parent_idx]
        child_joint = joints_2d[child_idx]

        # Skip if NaN in joint
        if np.isnan(parent_joint).any() or np.isnan(child_joint).any():
            continue

        # Draw line between parent and child joints
        parent_x, parent_y = parent_joint.astype(int)
        child_x, child_y = child_joint.astype(int)
        cv2.line(img, (parent_x, parent_y), (child_x, child_y), (255, 0, 0), 2)  # Blue line

# Save the output image
cv2.imwrite(f"visualize_{clip_name}_{frame_num}.jpg", img)

True

## 1. Genereate video with

In [22]:
output_video_path = f"/data/MHL/fifa-skeletal/outputs/{clip_name}_video.mp4"
output_image_path = f"/data/MHL/fifa-skeletal/outputs/{clip_name}_overlay"
# Create video capture object
cap = cv2.VideoCapture(video_path)

# Get video frame size and FPS information
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Create video writer object (save to output_video_path)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
frame_num = 0

while True:
    ret, img = cap.read()
    if not ret:
        break

    R = camera_params["R"][frame_num]
    t = camera_params["t"][frame_num]
    k = camera_params["k"][frame_num]
    f = camera_params["K"][frame_num][0, 0]
    principal_points = camera_params["K"][frame_num][:2, 2]
    img_shape = img.shape

    # Draw soccer field lines
    for line in SOCCER_FIELD_LINES:
        line = project_points(line, R=R, t=t, f=f, principal_points=principal_points, k=k, img_shape=img.shape)
        line = line.astype(np.int32)
        cv2.polylines(img, [line], isClosed=False, color=(0, 0, 255), thickness=2)  # Red lines

    # Visualize SMPL joints on the image (e.g., project SMPL joint coordinates to 2D)
    for smpl_seq in smpl_seqs:
        joints = smpl_seq.joints[frame_num]  # Joint coordinates of the first frame (e.g., body, hands)
        if np.isnan(joints).all():  # If joints contain NaN values
            continue  # Skip frames with NaN values

        # Project joint 3D coordinates to 2D image coordinates
        joints_2d = project_points(joints, R=R, t=t, f=f, principal_points=principal_points, k=k)

        # Visualize joint positions (draw a circle for each joint)
        for joint in joints_2d:
            x, y = joint.astype(int)
            cv2.circle(img, (x, y), 5, (0, 255, 0), -1)  # Green circles for joints

        # Draw skeleton joints connections (parent-child relationships)
        skeleton = smpl_seq.smpl_layer.skeletons()["body"]  # SMPL body joint connections
        for parent_idx, child_idx in skeleton.T:  # Parent-child relationships
            if parent_idx == -1:  # Skip root joint as it has no parent
                continue

            parent_joint = joints_2d[parent_idx]
            child_joint = joints_2d[child_idx]

            # Skip joints that contain NaN values
            if np.isnan(parent_joint).any() or np.isnan(child_joint).any():
                continue

            # Check if the coordinates are within the image boundaries
            if not (0 <= parent_joint[0] < frame_width and 0 <= parent_joint[1] < frame_height) or \
               not (0 <= child_joint[0] < frame_width and 0 <= child_joint[1] < frame_height):
                continue  # Skip joints outside the image range

            parent_x, parent_y = parent_joint.astype(int)
            child_x, child_y = child_joint.astype(int)

            # Draw line connecting parent and child joints
            cv2.line(img, (parent_x, parent_y), (child_x, child_y), (255, 0, 0), 2)  # Blue line

    frame_num += 1
    # Save the processed frame to the output video
    out.write(img)
    # Save the frame image
    cv2.imwrite(f"{output_image_path}/{frame_num:06d}.jpg", img)

# Release video capture and writer objects
cap.release()
out.release()

# Print message
print(f"The output video has been saved to {output_video_path}.")

The output video has been saved to /data/MHL/fifa-skeletal/outputs/ARG_CRO_220001_video.mp4.
